In [295]:
#def my_likelihood():
#    return


In [296]:
##Priors Info

# k1 = uniform(1.7*10**-9,1.06*10**-9)##...?
# GA1 = uniform(1.4,3.6)
# GA2 = uniform(0,8)
# GA3 = uniform(0.5,7.5)
# d1 = uniform(229.44,954.72)
#d2 = uniform(d1 + 487.610 , 1652.93)

In [297]:
#Conversion for scaling

MeV = 1 
GeV = 1000*MeV
fm  = 1
c   = 1
metres = (10**15)*fm
km = 1000*metres
second = 2.99*(10**8)*metres
kg = (second**2)/((1.6*(10**(-13)))*((metres)**2))
G = 6.67*10**(-11)*((metres)**3)/(kg*(second)**2)
mbar = 1.66*(10**-27)*kg

dp = 10**-12
mbar = 1.66*(10**-27)*kg
pi = math.pi
Rs = ((2*G*(1.98847*(10**30)*kg))/((c**2)))
K1 =  ((4*pi*(Rs**3))/(1.98847*((10**30)*kg)*(c**2)))*(GeV/(fm)**3)

In [298]:
import numpy as np
import scipy
import matplotlib.pyplot as plot
import math
from scipy.integrate import odeint
from scipy.stats import uniform

In [319]:
#SFHx
class TOV_Solver:
    def __init__(self,Pc,p1,p2,k1,GA1,GA2,GA3):
        self.Pc = Pc
        self.p1 = p1
        self.p2 = p2
        self.k1 = k1
        self.GA1 = GA1
        self.GA2 = GA2
        self.GA3 = GA3
    
    
    def piecewise_polytrope_SFHx(self,p):
    
        k2 = self.p1/((self.p1/self.k1)**(1/self.GA1) + self.p1/(self.GA1-1) - self.p1/(self.GA2-1))**self.GA2
        k3 = self.p2/((self.p2/k2)**(1/self.GA2) + self.p2/(self.GA2-1) - self.p2/(self.GA3-1))**self.GA3

        condition1 = (p < self.p1)
        condition2 = (p >= self.p1) & (p < self.p2)
        condition3 = (p >= self.p2) 
        condition = [condition1, condition2, condition3]
        Polytrope1 = lambda p: (mbar*(GeV*p/self.k1)**(1/self.GA1) + (p*GeV/(self.GA1-1)))/GeV
        Polytrope2 = lambda p: (mbar*(GeV*p/k2)**(1/self.GA2) + (p*GeV/(self.GA2-1)))/GeV
        Polytrope3 = lambda p: (mbar*(GeV*p/k3)**(1/self.GA3) + (p*GeV/(self.GA3-1)))/GeV
        function_list = [Polytrope1 , Polytrope2 , Polytrope3]
        d = (np.piecewise(p, condition, function_list))
    
        return (d)

    def R0(self,k,GA):
            dc = (mbar*(self.Pc*(GeV)/k)**(1/GA) + (self.Pc*(GeV)/(GA - 1)))/(GeV)
            P = self.Pc - dp 
            d = (mbar*(P*GeV/k)**(1/GA) +(P*GeV/(GA - 1)))/GeV
            nc = (((self.Pc*GeV)/k)**(1/GA))/GeV
            n = (((P*GeV)/k)**(1/GA))/GeV
            return  (((3/(2*pi*(3*self.Pc + dc)))*((math.log((dc + self.Pc)*n) - ((math.log((d + P)*nc)))))))**(0.5)

    def M0(self,k,GA):
            dc = (mbar*(self.Pc*(GeV)/k)**(1/GA) + (self.Pc*(GeV)/(GA - 1)))/(GeV)
            P = self.Pc - dp
            d = (mbar*(P*GeV/k)**(1/GA) +(P*GeV/(GA - 1)))/GeV
            nc = (((self.Pc*GeV)/k)**(1/GA))/GeV
            n = (((P*GeV)/k)**(1/GA))/GeV
            return ((((((math.log((dc + self.Pc)*n) - ((math.log((d + P)*nc)))))*(2*dc/(3*self.Pc + dc))))))*self.R0(k,GA)
    
        
    def f(self,x,P):
        m,r = x
        d = self.piecewise_polytrope_SFHx(P)
        #d = (mbar*((P*(GeV))/k)**(1/GA) + (P*(GeV))/(GA - 1))/(GeV)
        dmdp = -(2*K1*(r**3)*d*(r-m))/((d + P)*(m + K1*P*(r**3)))
        drdp = -(2*r*(r - m))/((d + P)*(m + K1*P*(r**3)))
        return [dmdp , drdp]  
            
            
    def compute_MR(self):
        k2 = self.p1/((self.p1/self.k1)**(1/self.GA1) + self.p1/(self.GA1-1) - self.p1/(self.GA2-1))**self.GA2
        k3 = self.p2/((self.p2/k2)**(1/self.GA2) + self.p2/(self.GA2-1) - self.p2/(self.GA3-1))**self.GA3
        
        if self.Pc < self.p1:
            Ro =  self.R0(k = self.k1, GA = self.GA1)
            Mo =  self.M0(k = self.k1, GA = self.GA1)

        elif (self.Pc >= self.p1) & (self.Pc < self.p2):
            Ro =  self.R0(k = k2, GA = self.GA2)
            Mo =  self.M0(k = k2, GA = self.GA2)
        
        elif (self.Pc >= self.p2):
            Ro =  self.R0(k = k3, GA = self.GA3)
            Mo =  self.M0(k = k3, GA = self.GA3)

        R = []
        M = []
        x_i = [Mo,Ro]
                         
        P0 = self.Pc/GeV - dp
            
        P1 = np.linspace(1e-12, P0, 1000)
        solve = odeint(self.f,x_i, P1[::-1])
                          
        m = solve[:,0]
        r = solve[:,1]

        M = (m[-1])
        R = (r[-1])*(Rs/km)
        return "M:",M ,"R:",R

In [320]:
A = TOV_Solver(Pc = 100, p1 = 6.32840085e+01/GeV, p2 = 5.99723682e+02/GeV, k1 = 4.74781083e-09*(GeV)**(1 - 3.88632843) , GA1 = 3.88632843, GA2 = 2.71342162, GA3 = 1.53107396)
A.compute_MR()

 lsoda--  at t (=r1), too much accuracy requested    
       for precision of machine..  see tolsf (=r2)   


/tmp/ipykernel_10199/1820029321.py:51: RuntimeWarning: invalid value encountered in scalar divide
  dmdp = -(2*K1*(r**3)*d*(r-m))/((d + P)*(m + K1*P*(r**3)))
/tmp/ipykernel_10199/1820029321.py:52: RuntimeWarning: invalid value encountered in scalar divide
  drdp = -(2*r*(r - m))/((d + P)*(m + K1*P*(r**3)))


('M:', 1.69759663307e-313, 'R:', 1.376683330074506e-309)

      in above,  r1 =  0.9989989989890D-01   r2 =                  NaN


In [285]:
##TOV-Solver

In [374]:
class TOV_Solver():
    """
    Class to Solve the TOV equation
    """
    
    pscale  = (GeV/fm**3)
    E_unit =  MeV / fm**3
    
    def __init__(self, p1 ,p2,k1 ,GA1 ,GA2 ,GA3 ,scode='odeint' , E_P=None, eos_object=None ):
        """
        Arguments:
        ---------
        scode: type of solver (odeint or ivp)
               string-like
        E_P: Method that returns Energy density (GeV/fm^3) 
               for a given pressure (GeV/fm^3)
               function-like
        eos_object: defines the background equation of state
                 class object like
        """
        
        self.scode = scode
        self.p1 = p1
        self.p2 = p2
        self.k1 = k1
        self.GA1 = GA1
        self.GA2 = GA2
        self.GA3 = GA3
        
        if eos_object is None:
            self.E_P = E_P
        else:
            self.eos_object = eos_object
            
            self.E_P = lambda P: eos_object.E_P(P*self.pscale)/self.pscale
    
    def piecewise_polytrope_SFHx(self,p):
    
        k2 = self.p1/((self.p1/self.k1)**(1/self.GA1) + self.p1/(self.GA1-1) - self.p1/(self.GA2-1))**self.GA2
        k3 = self.p2/((self.p2/k2)**(1/self.GA2) + self.p2/(self.GA2-1) - self.p2/(self.GA3-1))**self.GA3

        condition1 = (p < self.p1)
        condition2 = (p >= self.p1) & (p < self.p2)
        condition3 = (p >= self.p2) 
        condition = [condition1, condition2, condition3]
        Polytrope1 = lambda p: (mbar*(GeV*p/self.k1)**(1/self.GA1) + (p*GeV/(self.GA1-1)))/GeV
        Polytrope2 = lambda p: (mbar*(GeV*p/k2)**(1/self.GA2) + (p*GeV/(self.GA2-1)))/GeV
        Polytrope3 = lambda p: (mbar*(GeV*p/k3)**(1/self.GA3) + (p*GeV/(self.GA3-1)))/GeV
        function_list = [Polytrope1 , Polytrope2 , Polytrope3]
        d = (np.piecewise(p, condition, function_list))
        return (d)
    
    
    def dF(self, P, y):
        """
        Return RHS of TOV equation
        
        Arguments
        ---------
        P: pressure
           float
        y: list with mass and radius
           list-like
        """
        
        m, r = y[0], y[1]
        e = self.piecewise_polytrope_SFHx(P)
        denom = (e+P)*(m+K1*P*r**3)
        dy0 = -2*K1*r**3*e*(r-m)/denom
        dy1 = -2*r*(r-m)/denom
        return [dy0, dy1]
        
    
    
    def tsolve(self, pct=None, scode=None, rtol=1e-16, Ptol = 1e-10):
        """
        Return solution of TOV equation using central pressure
        
        Arguments
        ---------
        pct: central pressure (in GeV/fm^3)
             float-like
        solver: type of solver: ivp or ode
             string-like
        rtol: approximation of zero radius
        """
        
        if scode is None:
            scode = self.scode

        dP = -np.sqrt(rtol) * pct
        P1 = pct + dP
        E1 = self.piecewise_polytrope_SFHx(P1)

        #Evaluate at midpoint
        P = (pct + P1) / 2
        E = self.piecewise_polytrope_SFHx(P)
        E0 = self.piecewise_polytrope_SFHx(pct)

        
        r1 = np.sqrt(6 / E0 * np.log(
         (E + 3 * pct) * (E + P1) / (E + pct) / (E + 3 * P1)) / K1)
        M1 = K1 * E * r1**3 / 3.
        
        ps = np.linspace(Ptol, P, 1000)

        if scode == 'ivp':
            sol = solve_ivp(self.dF, [P, Ptol], [M1, r1],
                            method='Radau',
                            dense_output=True)
            mrs = np.array([sol.sol(_p) for _p in ps[::-1]])

        if scode == 'odeint':
            dF_int = lambda x, y: self.dF(y, x)
            sol = odeint(dF_int, [M1, r1], ps[::-1])
            mrs = sol
        
        else:
            raise NotImplementedError
            """
            t_ = tov.TOV(_eos, rtol=1e-6)
            M, R, k2 = t_.get_MRk(pct * pscale)
            sol = [M / u.M0, R / u.km]
            """
        
        return mrs, ps
    

In [375]:
A = TOV_Solver(p1 = 6.32840085e+01/GeV, p2 = 5.99723682e+02/GeV, k1 = 4.74781083e-09*(GeV)**(1 - 3.88632843) , GA1 = 3.88632843, GA2 = 2.71342162, GA3 = 1.53107396)
B = A.tsolve(pct = 100, scode = "odeint" )

/tmp/ipykernel_10199/849174835.py:46: RuntimeWarning: invalid value encountered in power
  Polytrope1 = lambda p: (mbar*(GeV*p/self.k1)**(1/self.GA1) + (p*GeV/(self.GA1-1)))/GeV


In [376]:
P1_t = 5.39550530e+01
P2_t = 7.97522989e+02

In [377]:
#SFHx
class TOV_Solver:
    def __init__(self,Pc):
        self.Pc = Pc
        
    
    
#     def piecewise_polytrope_SFHx(self,p):
    
#         k2 = self.p1/((self.p1/self.k1)**(1/self.GA1) + self.p1/(self.GA1-1) - self.p1/(self.GA2-1))**self.GA2
#         k3 = self.p2/((self.p2/k2)**(1/self.GA2) + self.p2/(self.GA2-1) - self.p2/(self.GA3-1))**self.GA3

#         condition1 = (p < self.p1)
#         condition2 = (p >= self.p1) & (p < self.p2)
#         condition3 = (p >= self.p2) 
#         condition = [condition1, condition2, condition3]
#         Polytrope1 = lambda p: (mbar*(GeV*p/self.k1)**(1/self.GA1) + (p*GeV/(self.GA1-1)))/GeV
#         Polytrope2 = lambda p: (mbar*(GeV*p/k2)**(1/self.GA2) + (p*GeV/(self.GA2-1)))/GeV
#         Polytrope3 = lambda p: (mbar*(GeV*p/k3)**(1/self.GA3) + (p*GeV/(self.GA3-1)))/GeV
#         function_list = [Polytrope1 , Polytrope2 , Polytrope3]
#         d = (np.piecewise(p, condition, function_list))
    
#         return (d)

    def R0(self,k,GA):
            dc = (mbar*(self.Pc*(GeV)/k)**(1/GA) + (self.Pc*(GeV)/(GA - 1)))/(GeV)
            P = self.Pc - dp 
            d = (mbar*(P*GeV/k)**(1/GA) +(P*GeV/(GA - 1)))/GeV
            nc = (((self.Pc*GeV)/k)**(1/GA))/GeV
            n = (((P*GeV)/k)**(1/GA))/GeV
            return  (((3/(2*pi*(3*self.Pc + dc)))*((math.log((dc + self.Pc)*n) - ((math.log((d + P)*nc)))))))**(0.5)

    def M0(self,k,GA):
            dc = (mbar*(self.Pc*(GeV)/k)**(1/GA) + (self.Pc*(GeV)/(GA - 1)))/(GeV)
            P = self.Pc - dp
            d = (mbar*(P*GeV/k)**(1/GA) +(P*GeV/(GA - 1)))/GeV
            nc = (((self.Pc*GeV)/k)**(1/GA))/GeV
            n = (((P*GeV)/k)**(1/GA))/GeV
            return ((((((math.log((dc + self.Pc)*n) - ((math.log((d + P)*nc)))))*(2*dc/(3*self.Pc + dc))))))*self.R0(k,GA)
    
        
    def f1(self,x,P):
        m,r = x
        k = 4.74781083e-09
        GA = 3.86298419e+00
        #d = self.piecewise_polytrope_SFHx(P)
        d = (mbar*((P*(GeV))/k)**(1/GA) + (P*(GeV))/(GA - 1))/(GeV)
        dmdp = -(2*K1*(r**3)*d*(r-m))/((d + P)*(m + K1*P*(r**3)))
        drdp = -(2*r*(r - m))/((d + P)*(m + K1*P*(r**3)))
        return [dmdp , drdp]  
            
    def f2(self,x,P):
        m,r = x
        k = 8.177335968849595e-06
        GA = 2.63557330e+00
        #d = self.piecewise_polytrope_SFHx(P)
        d = (mbar*((P*(GeV))/k)**(1/GA) + (P*(GeV))/(GA - 1))/(GeV)
        dmdp = -(2*K1*(r**3)*d*(r-m))/((d + P)*(m + K1*P*(r**3)))
        drdp = -(2*r*(r - m))/((d + P)*(m + K1*P*(r**3)))
        return [dmdp , drdp] 
    
    def f3(self,x,P):
        m,r = x
        k = 1.2701514501504945
        GA = 1.53329317e+00
        #d = self.piecewise_polytrope_SFHx(P)
        d = (mbar*((P*(GeV))/k)**(1/GA) + (P*(GeV))/(GA - 1))/(GeV)
        dmdp = -(2*K1*(r**3)*d*(r-m))/((d + P)*(m + K1*P*(r**3)))
        drdp = -(2*r*(r - m))/((d + P)*(m + K1*P*(r**3)))
        return [dmdp , drdp] 
            
    def compute_MR(self):
        
        
        if self.Pc < p1_t:
            Ro =  self.R0(k = 4.74781083e-09, GA = 3.86298419e+00)
            Mo =  self.M0(k = 4.74781083e-09, GA = 3.86298419e+00)

        elif (self.Pc >= p1_t) & (self.Pc < p2_t):
            Ro =  self.R0(k = 8.177335968849595e-06, GA = 2.63557330e+00)
            Mo =  self.M0(k = 8.177335968849595e-06, GA =2.63557330e+00)
        
        elif (self.Pc >= p2_t):
            Ro =  self.R0(k = 1.2701514501504945, GA = 1.53329317e+00)
            Mo =  self.M0(k = 1.2701514501504945, GA = 1.53329317e+00)

        R = []
        M = []
        x_i = [Mo,Ro]
                         
        P0 = self.Pc/GeV - dp
            
        P1 = np.linspace(P1_t, P0, 1000)
        solve1 = odeint(self.f1,x_i, P1[::-1])
        
        m1 = solve1[:,0]
        r1 = solve1[:,1]
        M1 = []
        R1 = []
        M1 = (m1[-1])
        R1 = (r1[-1])#*(Rs/km)
        x_i_1 = [M1,R1]
        
        P2 = np.linspace(P2_t, P1_t, 1000)
        solve2 = odeint(self.f2,x_i_1, P2[::-1])
        
        m2 = solve2[:,0]
        r2 = solve2[:,1]
        M2 = []
        R2 = []
        M2 = (m2[-1])
        R2 = (r2[-1])#*(Rs/km)
        x_i_2 = [M2,R2]
        
        P3 = np.linspace(1e-12, P2_t, 1000)
        solve3 = odeint(self.f3,x_i_2, P3[::-1])
                          
        m = solve3[:,0]
        r = solve3[:,1]

        M = (m[-1])
        R = (r[-1])*(Rs/km)
        return "M:",M ,"R:",R

In [379]:
A = TOV_Solver(Pc = 10)
A.compute_MR()

 lsoda--  at t (=r1), too much accuracy requested    
       for precision of machine..  see tolsf (=r2)   
      in above,  r1 =  0.6399905205106D-01   r2 =                  NaN


/tmp/ipykernel_10199/642533999.py:48: RuntimeWarning: invalid value encountered in scalar divide
  dmdp = -(2*K1*(r**3)*d*(r-m))/((d + P)*(m + K1*P*(r**3)))
/tmp/ipykernel_10199/642533999.py:49: RuntimeWarning: invalid value encountered in scalar divide
  drdp = -(2*r*(r - m))/((d + P)*(m + K1*P*(r**3)))


('M:', 6.9211014718135e-310, 'R:', 2.0535386002176e-309)